## IMPLEMENT A RNNS LANGUAGE MODEL FOR PREDICTING WORDS

### STEP 1: Import necessary libraries

In [ ]:
import torch 
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import os
import requests
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from utils.build_vocab import Vocab
from utils.Trainer import RNNTrainer

%matplotlib inline

### STEP 2: Download and preprocessing dataset

2.1. Download data

In [ ]:
print(os.getcwd())

In [ ]:
# Download dataset from a given url
url = "https://www.gutenberg.org/cache/epub/84/pg84.txt"
os.makedirs("../data", exist_ok=True)

response = requests.get(url)
filename = "../data/gutenberg_book.txt"

with open(filename, "w", encoding="utf-8") as f:
    f.write(response.text)

2.2. Preprocess data

In [ ]:
# Preprocess the data, the trained tokens will be separate words
import re
with open(filename, "r", encoding="utf-8") as f:
    text = f.read()

text = re.sub('[^A-Za-z]+', ' ', text).lower()
words = text.split()

vocab = Vocab(words)
encoded = vocab[words]

2.3. Explore data

In [ ]:
# Print the 10 most frequent words
freqs = vocab.token_freqs
for item in freqs[:10]:
    print(item)

In [ ]:
# Show the distribution of words in text
freq = [freq for word, freq in freqs]
plt.figure(figsize=(8, 5))
plt.xlabel('token: x')
plt.ylabel('frequency: n(x)')
plt.xscale('log')
plt.yscale('log')
plt.title('Distribution of words', fontweight='bold')
plt.plot(freq)

### STEP 3: Build a specialize Dataset Class

In [ ]:
class WordDataset(Dataset):
    def __init__(self, data, num_steps):
        self.data = data
        self.num_steps = num_steps

    # Number of samples
    def __len__(self):
        return len(self.data) - self.num_steps
    
    # Method to get samples based on index
    def __getitem__(self, index):
        x = self.data[index: index + self.num_steps]
        y = self.data[index + 1: index + self.num_steps + 1]
        return torch.tensor(x), torch.tensor(y)

Create dataloader which supports training

In [ ]:
num_steps = 10
num_epochs = 10
batch_size = 128
train_data, val_data = train_test_split(encoded, test_size=0.1, random_state=42)

train_dataset = WordDataset(train_data, num_steps)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

val_dataset = WordDataset(val_data, num_steps)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

### STEP 4: Build a language model based on RNNs

In [ ]:
class LMRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dims, num_hiddens, num_layers=1):
        super(LMRNN, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dims)
        self.rnn = nn.RNN(embedding_dims, num_hiddens, num_layers)
        self.fc = nn.Linear(num_hiddens, vocab_size)

    def forward(self, inputs, state=None):
        # input -> shape: (num_steps, batch_size)
        # x -> shape: (num_steps, batch_size, embedding_dims)
        x = self.embed(inputs)
        # out -> shape: (num_steps, batch_size, num_hiddens)
        # hn -> shape: (num_layers, batch_size, num_hiddens)
        out, hn = self.rnn(x, state)
        # y -> shape: (num_steps, batch_size, vocab_size)
        y = self.fc(out)
        return y, hn

### STEP 5: Training model

5.1. Set up parameters

In [ ]:
vocab_size = len(vocab)
embedding_dims = 128
num_hiddens = 256
num_layers = 5
epochs = 10
gradient_clip_val = 1.0

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

rnn_model = LMRNN(vocab_size, embedding_dims, num_hiddens, num_layers).to(device)
optimizer = optim.Adam(rnn_model.parameters(), lr=0.25)
criterion = nn.CrossEntropyLoss()
lr = 0.05

5.2. Create a trainer for training model

In [ ]:
rnn_trainer = RNNTrainer(rnn_model, vocab_size, train_loader, val_loader, lr, num_epochs, gradient_clip_val)

5.3. Start traing

In [ ]:
train_ppl, val_ppl = rnn_trainer.train_ppl, rnn_trainer.val_ppl
rnn_trainer.fit()

5.4. Showing perplexity

In [ ]:
rnn_trainer.plot()
print(f'Training loss at the last epoch: {train_ppl[-1]}')
print(f'Testing loss at the last epoch: {val_ppl[-1]}')

### STEP 6: Build functions for predicting words

6.1. Build a function for predicting the next word

In [ ]:
def predict_next_word(model, start_text):
    model.eval()
    words = start_text.lower().split()
    inputs = torch.tensor(vocab[words], dtype=torch.long).to(device)
    # Converts inputs to (num_steps, batch_size=1)
    inputs = inputs.reshape(-1, 1)

    with torch.no_grad():
        y, _ = model(inputs)
        last_word_digits = y[-1, 0, :]
        predicted_idx = torch.argmax(last_word_digits).item()
        predicted_word = vocab.to_tokens(predicted_idx)
    return predicted_word

6.2. Build a function for predicting a sequence

In [ ]:
def predict_next_sequence(model, start_text, seq_len):
    model.eval()
    words = start_text.lower().split()
    inputs = torch.tensor(vocab[words], dtype=torch.long).to(device)
    inputs = inputs.reshape(-1, 1)

    with torch.no_grad():
        for _ in range(seq_len):
            out, _ = model(inputs)
            next_word_idx = torch.argmax(out[-1, 0, :]).item()
            next_word = vocab.to_tokens(next_word_idx)
            words.append(next_word)
            # Take 'seq_len' words in 'words' to feed into the model
            inputs = torch.tensor([vocab[w] for w in words[-num_steps:]], dtype=torch.long).reshape(-1, 1).to(device)
    
    return " ".join(words)

6.3. Implementing predictions

In [ ]:
prompt = "I was traveling around the city"
print(predict_next_word(rnn_model, prompt))
print(predict_next_sequence(rnn_model, prompt, 20))

### STEP 7: Testing LSTM and GRU Model

7.1. Implement LSTM

7.1.1. Build a LSTM language model

In [ ]:
class LMLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dims, num_hiddens, num_layers=1):
        super(LMLSTM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dims)
        self.lstm = nn.LSTM(embedding_dims, num_hiddens, num_layers)
        self.fc = nn.Linear(num_hiddens, vocab_size)

    def forward(self, inputs, state=None):
        # input -> shape: (num_steps, batch_size)
        # x -> shape: (num_steps, batch_size, embedding_dims)
        x = self.embed(inputs)
        # out -> shape: (num_steps, batch_size, num_hiddens)
        # hn -> shape: (num_layers, batch_size, num_hiddens)
        out, hn = self.lstm(x, state)
        # y -> shape: (num_steps, batch_size, vocab_size)
        y = self.fc(out)
        return y, hn

7.1.2. Build a trainer class for LSTM model

In [ ]:
lstm_model = LMLSTM(vocab_size, embedding_dims, num_hiddens, num_layers).to(device)
lstm_trainer = RNNTrainer(lstm_model, vocab_size, train_loader, val_loader, lr, num_epochs, gradient_clip_val)

7.1.3. Train LSTM model

In [ ]:
lstm_trainer.fit()

7.1.4. Plot perplexity

In [ ]:
lstm_trainer.plot()

7.1.5. Predicting words

In [ ]:
prompt = "I was traveling around the city"
print(predict_next_word(lstm_model, prompt))
print(predict_next_sequence(lstm_model, prompt, 20))

7.2. Implement GRU

7.2.1. Build a GRU language model

In [ ]:
class LMGRU(nn.Module):
    def __init__(self, vocab_size, embedding_dims, num_hiddens, num_layers=1):
        super(LMGRU, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dims)
        self.gru = nn.GRU(embedding_dims, num_hiddens, num_layers)
        self.fc = nn.Linear(num_hiddens, vocab_size)

    def forward(self, inputs, state=None):
        # input -> shape: (num_steps, batch_size)
        # x -> shape: (num_steps, batch_size, embedding_dims)
        x = self.embed(inputs)
        # out -> shape: (num_steps, batch_size, num_hiddens)
        # hn -> shape: (num_layers, batch_size, num_hiddens)
        out, hn = self.gru(x, state)
        # y -> shape: (num_steps, batch_size, vocab_size)
        y = self.fc(out)
        return y, hn

7.2.2. Build a trainer class for GRU model

In [ ]:
gru_model = LMGRU(vocab_size, embedding_dims, num_hiddens, num_layers).to(device)
gru_trainer = RNNTrainer(lstm_model, vocab_size, train_loader, val_loader, lr, num_epochs, gradient_clip_val)

7.2.3. Train GRU model

In [ ]:
gru_trainer.fit()

7.2.4. Plot perplexity

In [ ]:
gru_trainer.plot()

7.2.5. Predicting words

In [ ]:
prompt = "I was traveling around the city"
print(predict_next_word(gru_model, prompt))
print(predict_next_sequence(gru_model, prompt, 20))